In [159]:
import yfinance
import pandas as pd
import numpy as np

In [135]:
# SNP500 ETF: https://finance.yahoo.com/quote/IVV/

sandp500 = yfinance.download (tickers = "IVV", start = "2025-01-01", 
                              interval = "1d")
sandp500 = sandp500.reset_index()
sandp500.columns = sandp500.columns.droplevel(1) # 2. Spaltenebene wegschmeißen
sandp500

/var/folders/58/r64xvznn1tb799qqpxxyf0m00000gn/T/ipykernel_40375/2433210841.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sandp500 = yfinance.download (tickers = "IVV", start = "2025-01-01",
[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
0,2025-01-02,583.806824,590.226722,579.712380,588.497523,9400400
1,2025-01-03,590.922363,591.737285,585.615515,586.678886,5867300
2,2025-01-06,594.589478,598.823068,592.731080,595.434242,4188000
3,2025-01-07,587.901306,596.845491,585.943492,596.567199,3810700
4,2025-01-08,588.517456,589.690131,584.333543,587.821785,5303600
...,...,...,...,...,...,...
127,2025-07-09,627.030029,627.719971,623.890015,625.750000,3019800
128,2025-07-10,628.849976,629.869995,626.010010,627.179993,2556800
129,2025-07-11,626.440002,627.820007,624.520020,625.760010,2739200
130,2025-07-14,627.739990,628.140015,624.750000,626.169983,2590500


In [71]:
sandp500[['Date', 'Close']].to_csv("sandp500.csv")

### 1 Tag vor Liberation Day gekauft und bis 11.07. gehalten

In [136]:
investment = 100

sp500_april_first = sandp500[sandp500['Date'] == '2025-04-01']['Close'].item()
share = investment / sandp500[sandp500['Date'] == '2025-04-01']['Close'].item()

sp500_july_11 = sandp500[sandp500['Date'] == '2025-07-11']['Close'].item()

print(f"Investiert: {investment}")
print(f"Finaler Portfoliowert: {share * sp500_july_11}")
print(f"Gewinn: {round(sp500_july_11 * share - investment, 2)}")
print(f"+{round(((sp500_july_11 / sp500_april_first) - 1) * 100, 2)}%")

Investiert: 100
Finaler Portfoliowert: 111.46120531438214
Gewinn: 11.46
+11.46%


In [ ]:
share = investment / sandp500[sandp500['Date'] == '2025-04-01']['Close'].item()
sp500_100_euro = sandp500[sandp500['Date'] >= '2025-04-01']
sp500_100_euro['profit'] = round((((share * sp500_100_euro['Close']) / investment) - 1) * 100, 2)
sp500_100_euro.to_csv("worst_timing_investment.csv")

/var/folders/58/r64xvznn1tb799qqpxxyf0m00000gn/T/ipykernel_40375/1357449462.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_100_euro['profit'] = round((((share * sp500_100_euro['Close']) / investment) - 1) * 100, 2)


### Bei jedem >=2% Dip 2025 gekauft und bis 11.07. gehalten

In [174]:
# Vor 1 Tag vor Liberation Day gekauft und bis 11.07. gehalten
drop_threhshold = 0.02

sandp500_drops = sandp500.copy()
sandp500_drops['Close_change_pct'] = sandp500_drops['Close'].pct_change()
sandp500_drops['investment_share'] = investment / sandp500_drops['Close']
drop_days = sandp500_drops[sandp500_drops['Close_change_pct'] <= -drop_threhshold]
drop_days

Price,Date,Close,High,Low,Open,Volume,Close_change_pct,investment_share
44,2025-03-10,559.945740,568.671258,554.787924,566.753234,7935600,-0.026050,0.178589
58,2025-03-28,556.382690,567.109207,555.954036,566.341584,7466000,-0.020189,0.179732
62,2025-04-03,538.388916,548.756507,537.740915,545.905424,12111300,-0.047831,0.185739
63,2025-04-04,506.029938,526.605706,505.840557,524.482331,17517100,-0.060103,0.197617
67,2025-04-10,525.419373,534.261757,510.336464,533.015648,10186400,-0.035131,0.190324
71,2025-04-16,526.496033,535.876715,521.112817,532.467377,6763400,-0.022307,0.189935
73,2025-04-21,514.772644,522.428731,509.239902,522.020034,4588600,-0.023247,0.194261


In [ ]:
drop_days_export = drop_days.copy()[['Date', 'Close', 'Close_change_pct']]
drop_days_export['Close_change_pct'] = round(drop_days_export['Close_change_pct'], 4) * 100
drop_days_export.to_csv('drop_days.csv')

In [175]:
total_investment = len(drop_days) * investment
total_shares = drop_days['investment_share'].sum()

print(f"Investiert: {total_investment}")
print(f"Finaler Portfoliowert: {total_shares * sp500_july_11}")
print(f"Gewinn: {round(total_shares * sp500_july_11 - total_investment, 2)}")
print(f"+{round(((total_shares * sp500_july_11 / total_investment) - 1) * 100, 2)}%")

Investiert: 700
Finaler Portfoliowert: 824.5183718939793
Gewinn: 124.52
+17.79%


In [179]:
sp500_since_jan = sandp500.copy()
sp500_since_jan['accumulated_investment_share'] = 0.0
sp500_since_jan['profit'] = 0.0
invested_money = 0

for idx in range(len(sp500_since_jan)):
    date = sp500_since_jan.at[idx, 'Date']
    close = sp500_since_jan.at[idx, 'Close']

    if idx > 0:
        prev_share = sp500_since_jan.at[idx - 1, 'accumulated_investment_share']
        sp500_since_jan.at[idx, 'accumulated_investment_share'] = prev_share
    else:
        prev_share = 0

    if date in list(drop_days['Date']):
        share = investment / close
        sp500_since_jan.at[idx, 'accumulated_investment_share'] = prev_share + share
        invested_money += investment

    current_share = sp500_since_jan.at[idx, 'accumulated_investment_share']
    sp500_since_jan.at[idx, 'profit'] = round(((current_share * close / invested_money) - 1) * 100, 2)


/var/folders/58/r64xvznn1tb799qqpxxyf0m00000gn/T/ipykernel_40375/1348872830.py:22: RuntimeWarning: invalid value encountered in scalar divide
  sp500_since_jan.at[idx, 'profit'] = round(((current_share * close / invested_money) - 1) * 100, 2)


In [180]:
sp500_since_jan

Price,Date,Close,High,Low,Open,Volume,accumulated_investment_share,profit
0,2025-01-02,583.806824,590.226722,579.712380,588.497523,9400400,0.000000,NaN
1,2025-01-03,590.922363,591.737285,585.615515,586.678886,5867300,0.000000,NaN
2,2025-01-06,594.589478,598.823068,592.731080,595.434242,4188000,0.000000,NaN
3,2025-01-07,587.901306,596.845491,585.943492,596.567199,3810700,0.000000,NaN
4,2025-01-08,588.517456,589.690131,584.333543,587.821785,5303600,0.000000,NaN
...,...,...,...,...,...,...,...,...
127,2025-07-09,627.030029,627.719971,623.890015,625.750000,3019800,1.316197,17.90
128,2025-07-10,628.849976,629.869995,626.010010,627.179993,2556800,1.316197,18.24
129,2025-07-11,626.440002,627.820007,624.520020,625.760010,2739200,1.316197,17.79
130,2025-07-14,627.739990,628.140015,624.750000,626.169983,2590500,1.316197,18.03


In [181]:
sp500_since_jan[sp500_since_jan['Date'] < '2025-07-12'].to_csv("btd_und_halten.csv")

### Bei jedem >=2% Dip 2025 gekauft und wieder verkauft bei Erholung

In [182]:
recovery_days = []
sandp500['Date'] = pd.to_datetime(sandp500['Date'])
drop_days['Date'] = pd.to_datetime(drop_days['Date'])

for _, row in drop_days.iterrows():
    drop_date = row['Date']
    drop_idx = sandp500[sandp500['Date'] == drop_date].index[0]

    prev_row = sandp500.iloc[drop_idx - 1]
    recovery_target = prev_row['Close']
    
    after = sandp500.iloc[drop_idx + 1:]
    recovered = after[after['Close'] >= recovery_target]

    if not recovered.empty:
        recovery_days.append(recovered.iloc[0])
    else:
        recovered.append("")

/var/folders/58/r64xvznn1tb799qqpxxyf0m00000gn/T/ipykernel_40375/2991664443.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_days['Date'] = pd.to_datetime(drop_days['Date'])


In [183]:
for day in recovery_days:
    print(day['Date'])

2025-03-24 00:00:00
2025-05-12 00:00:00
2025-05-02 00:00:00
2025-04-09 00:00:00
2025-04-24 00:00:00
2025-04-24 00:00:00
2025-04-22 00:00:00


In [184]:
drop_days

Price,Date,Close,High,Low,Open,Volume,Close_change_pct,investment_share
44,2025-03-10,559.945740,568.671258,554.787924,566.753234,7935600,-0.026050,0.178589
58,2025-03-28,556.382690,567.109207,555.954036,566.341584,7466000,-0.020189,0.179732
62,2025-04-03,538.388916,548.756507,537.740915,545.905424,12111300,-0.047831,0.185739
63,2025-04-04,506.029938,526.605706,505.840557,524.482331,17517100,-0.060103,0.197617
67,2025-04-10,525.419373,534.261757,510.336464,533.015648,10186400,-0.035131,0.190324
71,2025-04-16,526.496033,535.876715,521.112817,532.467377,6763400,-0.022307,0.189935
73,2025-04-21,514.772644,522.428731,509.239902,522.020034,4588600,-0.023247,0.194261


In [ ]:
df = drop_days.reset_index(drop=True)
profit = []

for idx, row in df.iterrows():
    print(f"bought on {row['Date']} for {row['Close']} with {100}, share: {row['investment_share']}")
    profit.append(row['investment_share'] * recovery_days[idx]['Close'] - investment)
    print(f"sold on {recovery_days[idx]['Date']} for {recovery_days[idx]['Close']}, profit: {profit[-1]}")
    print()

print()
accumulated = round(sum(profit), 2) + investment * len(drop_days)
print(f"Investiert: {investment * len(drop_days)}")
print(f"Gewinn: {round(sum(profit), 2)}")
print(f"+{round(((accumulated / (investment * len(drop_days))) - 1) * 100, 2)}%")

bought on 2025-03-10 00:00:00 for 559.9457397460938 with 100, share: 0.17858873262495897
sold on 2025-03-24 00:00:00 for 574.9248046875, profit: 2.675092223792703

bought on 2025-03-28 00:00:00 for 556.3826904296875 with 100, share: 0.17973240670512455
sold on 2025-05-12 00:00:00 for 583.8868408203125, profit: 4.943387144086728

bought on 2025-04-03 00:00:00 for 538.388916015625 with 100, share: 0.18573933642626814
sold on 2025-05-02 00:00:00 for 567.5678100585938, profit: 5.419668417193392

bought on 2025-04-04 00:00:00 for 506.0299377441406 with 100, share: 0.19761676640278564
sold on 2025-04-09 00:00:00 for 544.5496826171875, profit: 7.6121474244718

bought on 2025-04-10 00:00:00 for 525.4193725585938 with 100, share: 0.19032415861074517
sold on 2025-04-24 00:00:00 for 547.5103759765625, profit: 4.204451638392001

bought on 2025-04-16 00:00:00 for 526.4960327148438 with 100, share: 0.1899349544655755
sold on 2025-04-24 00:00:00 for 547.5103759765625, profit: 3.9913583305385316

boug

In [188]:
# ------------------------------------------------------
# Parameter: Kostenbasis nach Drop-Kurs oder fixer Einsatz?
# ------------------------------------------------------
use_drop_price_cost_basis = False  # True = share * drop_close; False = fester 'investment' pro Trade

# --- Schritt 1: Drop-Daten erweitern um Recovery-Infos ---
drop_df = drop_days.reset_index(drop=True).copy()

# Sicherstellen, dass Date-Spalten Timestamps sind
drop_df['Date'] = pd.to_datetime(drop_df['Date'])
recovery_dates = pd.to_datetime([d['Date'] for d in recovery_days])
recovery_closes = [d['Close'] for d in recovery_days]

drop_df['recovery_date'] = recovery_dates
drop_df['recovery_close'] = recovery_closes

if use_drop_price_cost_basis:
    drop_df['cost'] = drop_df['investment_share'] * drop_df['Close']
else:
    drop_df['cost'] = float(investment)  # fixer Einsatz pro Drop

# --- Schritt 2: Käufe & Verkäufe pro Datum gruppieren ---
buys_by_date = drop_df.groupby('Date').agg(
    shares_bought=('investment_share', 'sum'),
    cost_bought=('cost', 'sum')
)

sells_by_date = drop_df.groupby('recovery_date').agg(
    shares_sold=('investment_share', 'sum'),
    cost_sold=('cost', 'sum')
)

# Dict-Lookups
buy_shares = buys_by_date['shares_bought'].to_dict()
buy_costs  = buys_by_date['cost_bought'].to_dict()
sell_shares = sells_by_date['shares_sold'].to_dict()
sell_costs  = sells_by_date['cost_sold'].to_dict()

# --- Schritt 3: Timeseries-Tracking über sandp500 ---
sp500_since_jan = sandp500.copy().reset_index(drop=True)
sp500_since_jan['Date'] = pd.to_datetime(sp500_since_jan['Date'])

# Zielspalten (absolute & prozentuale Metriken)
cols_init = {
    'accumulated_investment_share': np.nan,
    'cost_basis_open': np.nan,
    'realized_profit_amt': np.nan,
    'realized_profit_pct': np.nan,
    'unrealized_profit_amt': np.nan,
    'unrealized_profit_pct': np.nan,
    'total_profit_amt': np.nan,
    'total_profit_pct': np.nan,
}
for c, v in cols_init.items():
    sp500_since_jan[c] = v

# Laufender State
shares_held = 0.0
cost_basis  = 0.0            # offene Kosten
realized_amt = 0.0           # realisierte Gewinne (absolut €)
cost_closed  = 0.0           # Summe der Kosten aller abgeschlossenen Trades
total_cost_ever_bought = 0.0 # kumulierte Kosten aller Käufe (historisch; sinkt NICHT bei Verkauf)

for idx in range(len(sp500_since_jan)):
    date  = sp500_since_jan.at[idx, 'Date']
    close = sp500_since_jan.at[idx, 'Close']

    # --- Käufe ---
    if date in buy_shares:
        sh   = buy_shares[date]
        cost = buy_costs[date]
        shares_held += sh
        cost_basis  += cost
        total_cost_ever_bought += cost  # historisch kumuliert

    # --- Verkäufe ---
    if date in sell_shares:
        sh   = sell_shares[date]
        cost = sell_costs[date]         # ursprüngliche Kosten der verkauften Shares
        proceeds = sh * close
        trade_profit = proceeds - cost

        realized_amt += trade_profit    # absolute Gewinne addieren
        cost_closed  += cost            # Basis der abgeschlossenen Trades akkumulieren

        shares_held -= sh               # Bestand reduzieren
        cost_basis  -= cost             # Kostenbasis offener Positionen reduzieren

    # --- Profit-Berechnungen ---
    # Unrealisiert (falls offene Positionen)
    if cost_basis > 0:
        unrealized_amt = shares_held * close - cost_basis
        unrealized_pct = unrealized_amt / cost_basis
    else:
        unrealized_amt = 0.0
        unrealized_pct = 0.0

    # Realisiert % (basierend auf abgeschlossenen Kosten)
    if cost_closed > 0:
        realized_pct = realized_amt / cost_closed
    else:
        realized_pct = 0.0

    # Gesamt (Realized + Unrealized) relativ zu *aller jemals eingesetzten Kosten*
    denom_total = total_cost_ever_bought if total_cost_ever_bought > 0 else np.nan
    total_profit_amt = realized_amt + unrealized_amt
    total_profit_pct = total_profit_amt / denom_total if denom_total > 0 else 0.0

    # --- In DataFrame schreiben ---
    sp500_since_jan.at[idx, 'accumulated_investment_share'] = shares_held
    sp500_since_jan.at[idx, 'cost_basis_open'] = cost_basis
    sp500_since_jan.at[idx, 'realized_profit_amt'] = realized_amt
    sp500_since_jan.at[idx, 'realized_profit_pct'] = realized_pct * 100.0
    sp500_since_jan.at[idx, 'unrealized_profit_amt'] = unrealized_amt
    sp500_since_jan.at[idx, 'unrealized_profit_pct'] = unrealized_pct * 100.0
    sp500_since_jan.at[idx, 'total_profit_amt'] = total_profit_amt
    sp500_since_jan.at[idx, 'total_profit_pct'] = total_profit_pct * 100.0


In [190]:
sp500_since_jan[['Date', 'total_profit_pct']]

Price,Date,total_profit_pct
0,2025-01-02,0.000000
1,2025-01-03,0.000000
2,2025-01-06,0.000000
3,2025-01-07,0.000000
4,2025-01-08,0.000000
...,...,...
127,2025-07-09,4.480519
128,2025-07-10,4.480519
129,2025-07-11,4.480519
130,2025-07-14,4.480519


In [192]:
sp500_since_jan[['Date', 'total_profit_pct']].to_csv("btd_und_verkaufe.csv")